In [1]:
!source activate /home/gilbee3/anaconda3/envs/akita

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
sns.set(palette='husl', context = 'poster', style='white', color_codes=True)
import pybedtools
import pysam

In [3]:
intervals = '/wynton/group/capra/projects/modern_human_3Dgenome/data/reference/genome_chunks_large.txt'

In [4]:
intervals

'/wynton/group/capra/projects/modern_human_3Dgenome/data/reference/genome_chunks_large.txt'

# variant specific

In [5]:
gagp = pd.read_table('/wynton/group/capra/data/hg38_ancestral/GAGP/all_chrs_hg38.txt', names=['chr','pos','gagp'])
gagp = gagp[gagp.chr != 'chrX']

In [6]:
new = pd.read_table('/wynton/group/capra/projects/modern_human_3Dgenome/data/tree_sequences/all.txt', names = ['chr','pos','hsmrca','ref','rsid'],header=0)

In [8]:
gagp['gagp'] = gagp['gagp'].str.upper()
new['hsmrca'] = new['hsmrca'].str.upper()
new['ref'] = new['ref'].str.upper()

In [9]:
chrs = ['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22']

In [10]:
gagp = gagp[gagp.chr.isin(chrs)]
new = new[new.chr.isin(chrs)]

In [11]:
new.shape

(3255340, 5)

In [13]:
new[new.ref == new.hsmrca]

,chr,pos,hsmrca,ref,rsid


In [14]:
gagp.shape

(172477379, 3)

In [15]:
gagp.shape[0]/new.shape[0]

52.98290777614627

In [16]:
new

,chr,pos,hsmrca,ref,rsid
0,chr1,51898,A,C,NaN
1,chr1,51928,A,G,NaN
2,chr1,52058,C,G,rs62637813
3,chr1,52238,G,T,rs2691277
4,chr1,54380,C,T,rs2691279
...,...,...,...,...,...
3255335,chr22,50758085,T,A,rs1229607216
3255336,chr22,50759324,C,A,rs6010078
3255337,chr22,50760478,A,G,rs6010079
3255338,chr22,50762027,T,G,NaN


In [17]:
gagp

,chr,pos,gagp
0,chr10,14583,T
1,chr10,14657,A
2,chr10,15029,A
3,chr10,15381,A
4,chr10,15479,G
...,...,...,...
179321487,chr14,91431132,C
179321488,chr14,91431161,G
179321489,chr14,91431263,C
179321490,chr14,91431301,G


In [18]:
m = gagp.merge(new, on=['chr','pos'], suffixes=['_gagp','_new'])

In [21]:
m['gagp'] = m['gagp'].apply(lambda x: x.upper())
m['hsmrca'] = m['hsmrca'].apply(lambda x: x.upper())
#m = m.drop(columns = ['allele','anc'])

In [22]:
m

,chr,pos,gagp,hsmrca,ref,rsid
0,chr10,113136,A,A,G,rs4881551
1,chr10,113359,T,T,A,rs4881552
2,chr10,113464,A,A,G,rs4880750
3,chr10,113997,C,C,G,rs7090289
4,chr10,114538,G,G,A,rs10903349
...,...,...,...,...,...,...
3086907,chr9,138127843,G,G,A,rs9410073
3086908,chr9,138136987,C,T,C,rs56009993
3086909,chr9,138137446,G,G,T,rs9695626
3086910,chr9,138139027,C,T,C,rs112830858


In [24]:
m[m.gagp == m.ref]

,chr,pos,gagp,hsmrca,ref,rsid
17,chr10,122137,C,A,C,rs779224767
63,chr10,186155,G,A,G,NaN
69,chr10,196960,C,T,C,rs954053741
70,chr10,201607,T,C,T,rs371707079
77,chr10,208248,G,A,G,rs564704259
...,...,...,...,...,...,...
3086877,chr9,138104983,C,T,C,rs372677914
3086878,chr9,138107857,C,T,C,rs148032981
3086893,chr9,138116221,C,T,C,rs755116475
3086908,chr9,138136987,C,T,C,rs56009993


In [60]:
same_anc = m[m.gagp == m.hsmrca]
same_anc

,chr,pos,ref,rsid,gagp,hsmrca
0,chr10,113136,G,rs4881551,A,A
1,chr10,113359,A,rs4881552,T,T
2,chr10,113464,G,rs4880750,A,A
3,chr10,113997,G,rs7090289,C,C
4,chr10,114538,A,rs10903349,G,G
...,...,...,...,...,...,...
3086905,chr9,138123110,A,rs9410163,G,G
3086906,chr9,138123160,T,rs915083768,C,C
3086907,chr9,138127843,A,rs9410073,G,G
3086909,chr9,138137446,T,rs9695626,G,G


In [61]:
same_anc[same_anc.ref == same_anc.gagp]

,chr,pos,ref,rsid,gagp,hsmrca


In [52]:
diffs = m[m.gagp != m.hsmrca]

In [53]:
diffs.shape

(223664, 6)

In [54]:
diffs

,chr,pos,ref,rsid,gagp,hsmrca
17,chr10,122137,C,rs779224767,C,A
63,chr10,186155,G,NaN,G,A
69,chr10,196960,C,rs954053741,C,T
70,chr10,201607,T,rs371707079,T,C
77,chr10,208248,G,rs564704259,G,A
...,...,...,...,...,...,...
3086877,chr9,138104983,C,rs372677914,C,T
3086878,chr9,138107857,C,rs148032981,C,T
3086893,chr9,138116221,C,rs755116475,C,T
3086908,chr9,138136987,C,rs56009993,C,T


In [55]:
diffs[diffs.ref == diffs.gagp]

,chr,pos,ref,rsid,gagp,hsmrca
17,chr10,122137,C,rs779224767,C,A
63,chr10,186155,G,NaN,G,A
69,chr10,196960,C,rs954053741,C,T
70,chr10,201607,T,rs371707079,T,C
77,chr10,208248,G,rs564704259,G,A
...,...,...,...,...,...,...
3086877,chr9,138104983,C,rs372677914,C,T
3086878,chr9,138107857,C,rs148032981,C,T
3086893,chr9,138116221,C,rs755116475,C,T
3086908,chr9,138136987,C,rs56009993,C,T


In [56]:
diffs[diffs.ref == diffs.hsmrca]

,chr,pos,ref,rsid,gagp,hsmrca


In [62]:
diffs[diffs.ref != diffs.hsmrca][diffs.ref != diffs.gagp]

,chr,pos,ref,rsid,gagp,hsmrca
358,chr10,414742,C,rs7081507,?,T
461,chr10,459378,C,10_505318,?,G
526,chr10,503173,T,rs1463669094,?,C
973,chr10,808725,G,rs964200847,?,A
3796,chr10,2542525,A,rs11251346,?,G
...,...,...,...,...,...,...
3086145,chr9,137411761,C,rs114989935,?,G
3086204,chr9,137472646,A,rs1207597310,?,G
3086466,chr9,137724910,T,9_140619362,C,G
3086620,chr9,137839809,A,rs5011581,?,T


In [63]:
6615+217049

223664